In [64]:
import datetime
import seaborn as sns
import pandas as pd
import numpy as np
import pysal
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import shape
from shapely.geometry import Point
import requests
import random
import overpy
import contextily as ctx

In [ ]:
 получить велодорожки GET /v1/datasets/{datasetId}/features

897 - велодорожки
674 - иск неровности

In [ ]:
def get_dict_objects(list_cat, api_key):
    url='https://apidata.mos.ru/v1/datasets/?api_key=%s'%(api_key)
    r = requests.get(url)
    if r.status_code==200:
        datasets=r.json()
#         print(datasets[0].keys())
        datasets[0]['ContainsGeodata']==True
        super_big_dict=dict()
        for i in range(len(datasets)):
            idi=datasets[i]['Id'] 
            if datasets[i]['ContainsGeodata'] and datasets[i]['CategoryId'] in list_cat and datasets[i]['IsArchive']==False :
                print(idi)
                url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(idi, api_key)
                r = requests.get(url)
                super_big_dict[idi]=r.json()
                #print(idi, super_big_dict[idi].keys())
                super_big_dict[idi]['Caption']=datasets[i]['Caption']
                super_big_dict[idi]['CategoryId']=datasets[i]['CategoryId']
                super_big_dict[idi]['DepartmentId']=datasets[i]['DepartmentId']
    else: print (r)    
    return super_big_dict

def get_df_attract(big_dict):
    df=pd.DataFrame()
    i=0
    for k,v in big_dict.items():
        df=pd.DataFrame(v['features'])
        df['Caption']=v['Caption']
        df['CategoryId']=v['CategoryId']
        df['DepartmentId']=v['DepartmentId']
        if i==0:
            df_fin=df.copy()
        else:
            df_fin=pd.concat([df_fin, df], axis=0)
        i+=1  
    df_fin['geometry']=df_fin['geometry'].apply(lambda x: shape(x))
    df_fin['dataset_id']=df_fin['properties'].apply(lambda x: x['DatasetId'])
    df_fin=df_fin.reset_index()
    return gpd.GeoDataFrame(df_fin, crs={'init': 'epsg:4326'}, geometry='geometry')


In [65]:
api_key='a2327bd3ff5aceb690bbb7b3199bfddf'

In [13]:
url='https://apidata.mos.ru/v1/datasets/897/features?api_key=%s'%(api_key)
r = requests.get(url)

In [14]:
r.json()

{'features': [{'geometry': {'coordinates': [[[37.157132239276,
       55.549413245372],
      [37.156178327272, 55.547646225529],
      [37.156360717485, 55.546946718418],
      [37.159311147402, 55.546040381917],
      [37.16187533922, 55.545492920173],
      [37.170498737528, 55.543594266524],
      [37.175712951853, 55.542395848739],
      [37.183543137178, 55.540482686296],
      [37.193942043317, 55.539470242686],
      [37.19995019151, 55.536914926877],
      [37.205915424359, 55.534627169332],
      [37.228918048871, 55.532168897892],
      [37.232716056836, 55.53159690172],
      [37.235226604475, 55.529321004418],
      [37.236969022613, 55.527222632844],
      [37.237767303178, 55.524978081809],
      [37.239346477521, 55.520094354792],
      [37.240591022504, 55.512583059127],
      [37.241749736798, 55.511682015275],
      [37.261705371869, 55.50510620324],
      [37.266597721113, 55.502987095463],
      [37.268528911603, 55.502670437337],
      [37.280716869367, 55.5030358

In [18]:

v=r.json()
df=pd.DataFrame(v['features'])

In [19]:
df['geometry']=df['geometry'].apply(lambda x: shape(x))
df['dataset_id']=df['properties'].apply(lambda x: x['DatasetId'])

In [27]:
df['bikeline_width'] = df['properties'].apply(lambda x: x['Attributes']['Width'])

In [29]:
df['bikeline_type'] = df['properties'].apply(lambda x: x['Attributes']['Type'])

In [35]:
from shapely import wkt

In [49]:
gdf_bike_lanes = gpd.GeoDataFrame(df,geometry = 'geometry',crs=4326)

In [63]:
gdf_bike_lanes.iloc[0]

geometry          (LINESTRING (37.157132239276 55.549413245372, ...
properties        {'DatasetId': 897, 'VersionNumber': 5, 'Releas...
type                                                        Feature
dataset_id                                                      897
bikeline_width                                                  1.8
bikeline_type     [Велопешеходная дорожка с совмещенным движение...
Name: 0, dtype: object

In [55]:
gdf_bike_lanes[['bikeline_width','geometry']].to_file('bike_lanes.geojson', driver='GeoJSON')

In [ ]:
def get_osm(api, city, tag, key=None): # query OpenStreetMap data
    if key is None:
        query="""area["name"=%s];node[%s](area);out;"""%('"'+city+'"','"'+tag+'"')
    else:
        query="""area["name"=%s];node[%s=%s](area);out;"""%('"'+city+'"','"'+tag+'"','"'+key+'"')
    print(query)
    result = api.query(query)
    amenities=set([i.tags[tag] for i in result.nodes])
    return amenities, result

api = overpy.Overpass()
moscow_tour, results1=get_osm(api, 'Москва', 'tourism')
moscow_railways, results=get_osm(api, 'Москва', 'railway', 'station')

In [ ]:
def get_amenities(api, city):
    query="""area["name"=%s];node["amenity"](area);out;"""%('"'+city+'"')
    print(query)
    result = api.query(query)
    amenities=set([i.tags['amenity'] for i in result.nodes])
    return amenities, result

def get_tourism(api, city):
    query="""area["name"=%s];node["tourism"](area);out;"""%('"'+city+'"')
    print(query)
    result = api.query(query)
    amenities=set([i.tags['tourism'] for i in result.nodes])
    return amenities, result


In [ ]:
#Open data MSK объекты

def get_dict_objects(list_cat, api_key):
    url='https://apidata.mos.ru/v1/datasets/?api_key=%s'%(api_key)
    r = requests.get(url)
    if r.status_code==200:
        datasets=r.json()
        print(datasets[0].keys())
        datasets[0]['ContainsGeodata']==True
        super_big_dict=dict()
        for i in range(len(datasets)):
            idi=datasets[i]['Id'] 
            if datasets[i]['ContainsGeodata'] and datasets[i]['CategoryId'] in list_cat and datasets[i]['IsArchive']==False :
                print(idi)
                url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(idi, api_key)
                r = requests.get(url)
                super_big_dict[idi]=r.json()
                #print(idi, super_big_dict[idi].keys())
                super_big_dict[idi]['Caption']=datasets[i]['Caption']
                super_big_dict[idi]['CategoryId']=datasets[i]['CategoryId']
                super_big_dict[idi]['DepartmentId']=datasets[i]['DepartmentId']
    else: print (r)    
    return super_big_dict
def get_df_attract(big_dict):
    df=pd.DataFrame()
    i=0
    for k,v in big_dict.items():
        df=pd.DataFrame(v['features'])
        df['Caption']=v['Caption']
        df['CategoryId']=v['CategoryId']
        df['DepartmentId']=v['DepartmentId']
        if i==0:
            df_fin=df.copy()
        else:
            df_fin=pd.concat([df_fin, df], axis=0)
        i+=1  
    df_fin['geometry']=df_fin['geometry'].apply(lambda x: shape(x))
    df_fin['dataset_id']=df_fin['properties'].apply(lambda x: x['DatasetId'])
    df_fin=df_fin.reset_index()
    return gpd.GeoDataFrame(df_fin, crs={'init': 'epsg:4326'}, geometry='geometry')
In [91]:
def get_add_cols(df): # add columns from atributes
    l=df['properties'].apply(lambda x: list(x['Attributes'].keys()))
    s=set()
    for v in l:
        s.update(v)
    cols=[i for i in s if 'name' in str.lower(i)  or 'hours' in str.lower(i) or 'disab' in str.lower(i) or 'paid' in str.lower(i) or 'site' in i or 'id' in i or 'time' in i]
    for name in cols:
        df[name]=df['properties'].apply(lambda x:  x['Attributes'][name] if  name in  x['Attributes'].keys() else None)
    df.drop(df.isnull().mean()[df.isnull().mean()==1].index.tolist(), axis=1, inplace=True)
    return df

def func_val(dict_names, col, dep_id):
    #print(dict_names[dep_id][col])
    return dict_names[dep_id][col]

def get_dict_names(fin_cols, df_attract):
    dic_dep_fields=dict()
    for cat in df_attract['DepartmentId'].unique():
        dic_dep_fields[cat]=get_full_column(df_attract[df_attract['DepartmentId']==cat])
    dict2=dict()
    for k, v in dic_dep_fields.items():
        dict2[k]=dict()
        for col in fin_cols:
            dict2[k][col]=get_val(col,v)
    return dict2 #dic_dep_fields

def get_val(fin_col, values):
    l=[i for i in values if fin_col in i]
    if len(l)==0:
        return None
    elif len(l)==1:
        return l[0]
    elif fin_col=='Name':
        l=[ i for i in l if ('Winter' not in i) and ('Organiz' not in i) and ('Balance' not in i)]
        return l[0]
    else: return l[0]

def get_full_column(df):
    return df.isnull().mean()[df.isnull().mean()<=0.5].index.tolist()

def get_fin_cols():
    return ['geometry',  'Caption', 'CategoryId', 'DepartmentId', 'dataset_id', 'global_id','Name','WorkingHours', 'DisabilityFriendly']


In [ ]:
#60662 - остановки

62525

Препятствия:
 - автобусные остановки +
 - мусорки +
 - туалетные модули +
 - фонарные столбы +
 - рекламные щиты +
 - киоски и 
 - дорожные работы на пешеходной части
    

In [98]:

def get_add_cols(df): # add columns from atributes
    l=df['properties'].apply(lambda x: list(x['Attributes'].keys()))
    s=set()
    for v in l:
        s.update(v)
    cols=[i for i in s] # if 'name' in str.lower(i)  or 'hours' in str.lower(i) or 'disab' in str.lower(i) or 'paid' in str.lower(i) or 'site' in i or 'id' in i or 'time' in i
    for name in cols:
        df[name]=df['properties'].apply(lambda x:  x['Attributes'][name] if  name in  x['Attributes'].keys() else None)
    df.drop(df.isnull().mean()[df.isnull().mean()==1].index.tolist(), axis=1, inplace=True)
    return df

In [66]:
url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(60662, api_key)
r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_bus_stops = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

In [127]:
gdf_bus_stops = gpd.GeoDataFrame(gdf_bus_stops, crs='epsg:4326', geometry='geometry')

In [129]:
gdf_bus_stops_fin = gdf_bus_stops[['geometry','global_id']]
gdf_bus_stops_fin['feature_type']='bus_stops'
gdf_bus_stops_fin['size_m2']=2
gdf_bus_stops_fin['start_install'] =2015 
gdf_bus_stops_fin['end_install']=2050

/Users/intra26/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [82]:
url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(1421, api_key)
r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_parking_machines = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

In [123]:
gdf_parking_machines=get_add_cols(gdf_parking_machines)

In [125]:
gdf_parking_machines_fin = gdf_parking_machines[['geometry','global_id']]

In [126]:
gdf_parking_machines_fin['feature_type']='parking machine'
gdf_parking_machines_fin['size_m2']=0.25
gdf_parking_machines_fin['start_install'] =2015 
gdf_parking_machines_fin['end_install']=2050

/Users/intra26/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [84]:
url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(1494, api_key)
r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_toilets = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

In [120]:
gdf_toilets=get_add_cols(gdf_toilets)

In [122]:
gdf_toilets.iloc[0]

geometry                                 POINT (37.561159890621 55.721990388237)
properties                     {'DatasetId': 1494, 'VersionNumber': 3, 'Relea...
type                                                                     Feature
Name                                                                    ПКТ №114
global_id                                                              436504857
PaidService                                                              платный
DisabilityFriendly                                               не приспособлен
ClarificationOfWorkingHours                                                     
WorkingHours                   [{'is_deleted': 0, 'DayOfWeek': 'понедельник',...
Balanceholder                                                            ООО ОРК
CloseFlag                                                              действует
HasPaymentTerminal                                                          есть
District                    

In [131]:
gdf_toilets_fin = gdf_toilets[['geometry','global_id']]
gdf_toilets_fin['feature_type']='toilets'
gdf_toilets_fin['size_m2']=8
gdf_toilets_fin['start_install'] =2015 
gdf_toilets_fin['end_install']=2050

In [86]:
#1122 рекламные щиты

url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(1122, api_key)
r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_advert = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

In [117]:
gdf_advert=get_add_cols(gdf_advert)

In [119]:
gdf_advert.iloc[0]

geometry                                                    POINT (37.55731 55.673092)
properties                           {'DatasetId': 1122, 'VersionNumber': 3, 'Relea...
type                                                                           Feature
global_id                                                                      3068494
ConstructionSize                                                                   6x3
Owner                                                             ООО «Гэллэри Сервис»
District                                                               район Черёмушки
TechCharasteristics                           электронная технология смены изображения
Longitude_WGS84                                                               37.55731
OldNumberOfExploitationPermission                                                     
ConstructionType                                                                   Щит
IsOnScheme                                 

In [133]:
gdf_advert_fin = gdf_advert[['geometry','global_id']]
gdf_advert_fin['feature_type']='advertising'
gdf_advert_fin['size_m2']=0.04
gdf_advert_fin['start_install'] =2015 
gdf_advert_fin['end_install']=gdf_advert['ContractExpiryDate']

In [96]:
# 	61762  все фонарные столбы!
url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(61762, api_key)
r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_lights = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [108]:
# киоски 619
# url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(619, api_key)
# r = requests.get(url)

# v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_kiosks = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

In [114]:
gdf_kiosks.ObjectType.value_counts()

киоск                                                                                      1371
передвижной торговый объект (тележки, лотки, палатки и иные специальные приспособления)     411
павильон                                                                                    271
елочный базар                                                                               230
бахчевой развал                                                                             198
торговый автомат                                                                            164
автомагазин                                                                                   1
Name: ObjectType, dtype: int64

In [134]:
gdf_kiosks.iloc[0]

geometry                                       POINT (37.759371 55.70241)
properties              {'DatasetId': 619, 'VersionNumber': 3, 'Releas...
type                                                              Feature
StartDateTrading                                               22.05.2017
Name                                        Цветы на Юных Ленинцев, вл.53
FacilityArea                                                            6
global_id                                                        20271656
PeriodOfPlacement                                с 1 января по 31 декабря
EndDateTrading                                                 01.04.2020
ContractNumber                                           НТО-01-02-003863
ContractEnd                                                    05.05.2022
ObjectPhoto                                                          None
ContractStatus                                                аннулирован
District                              

In [136]:
gdf_kiosks_fin = gdf_kiosks[gdf_kiosks.ObjectType.isin(['киоск','торговый автомат'])][['geometry','global_id']]
gdf_kiosks_fin['feature_type']='kiosk'
gdf_kiosks_fin['size_m2']=2
gdf_kiosks_fin['start_install']=gdf_kiosks['ContractBegin'] 
gdf_kiosks_fin['end_install']=gdf_kiosks['ContractEnd']


In [109]:
# дорожные работы 2527 нужна дата TS_from # нужно пересчитать!
# url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(2527, api_key)
# r = requests.get(url)

v =r.json()

df=pd.DataFrame(v['features'])

df['geometry']=df['geometry'].apply(lambda x: shape(x))

df = get_add_cols(df)

gdf_roadworks = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [143]:
gdf_obstacles = pd.concat([gdf_parking_machines_fin,gdf_kiosks_fin, gdf_toilets_fin, gdf_advert_fin, gdf_bus_stops_fin])

In [145]:
gdf_obstacles.to_file('gdf_obstacles_1.geojson', driver='GeoJSON')

In [ ]:
gdf_obstacles=pd.concat([gdf_obstacles, gdf_roadworks_fin, gdf_lumpposts_fin])

In [147]:
df_moscow_gis = pd.read_csv('/Users/intra26/Documents/данные москвы/objects_map', sep='\t')

df_bc = df_moscow_gis[df_moscow_gis['rubric']=='Бизнес-центр']

# df_bc['geometry']=get_points(df_bc['lat'],df_bc['lon'])

# gdf_bc = get_gdf(df_bc,'geometry',4326)

In [192]:
query="""area["name"=%s];node[~"building"~".*"](area);out;"""%("\'Москва\'")
print(query)
result = api.query(query)

area["name"='Москва'];node[~"building"~".*"](area);out;


In [226]:
api = overpy.Overpass()

query1="""area["name"='Москва'];
  // query part for: “buildings=* and use=office”
 // node["building"="office"](area);//
    way["building"="office"](area);
 
  relation["building"="office"](area);"""
print(query1)
result1 = api.query(query1)

area["name"='Москва'];
  // query part for: “buildings=* and use=commercial”
 // node["building"="office"](area);//
    way["building"="office"](area);
 
  relation["building"="office"](area);


In [234]:
df_bc=gpd.read_file('/Users/intra26/Documents/данные москвы/export.geojson') 

In [241]:
df_bc.shape

(1480, 131)

In [240]:
df_bc[df_bc.id=='way/127717485']

,id,@id,addr:housenumber,addr:street,building,building:levels,source:building:levels,addr:city,addr:country,name,...,int_name,building:min_level,building:parts,contact:vk,landuse,building:max_level,window:material,window:shape,material,geometry
957,way/127717485,way/127717485,12 к5,1-й Троицкий переулок,office,3,None,None,None,"Бизнес-центр ""Троицкий""",...,None,None,None,None,None,None,None,None,None,"POLYGON ((37.62116 55.77671, 37.62138 55.77668..."


In [250]:
df_bc['building:levels'].astype(float).median()

3.0

In [251]:
df_bc['capacity']=df_bc['building:levels'].astype(float).fillna(3)*200
df_bc['pedestrians'] = df_bc['capacity']*0.7

In [258]:
df_bc[['name','addr:street','building:levels','geometry','capacity','pedestrians']].to_file('bc_moscow_capacity.geojson',driver='GeoJSON')

In [ ]:
#  метро обработка. Добавляем весы на станции
df_metro = pd.read_csv('/Users/intra26/Desktop/coursera/станции_метро.csv')

df_metro['geometry']=get_points(df_metro['Latitude_WGS84'],df_metro['Longitude_WGS84'])

gdf_metro = get_gdf(df_metro,'geometry',4326)

gdf_metro['Caption']='metro_station'

gdf_metro['buf_200m'] = gdf_metro.to_crs('epsg:32637').geometry.buffer(200)

gdf_metro200= gpd.sjoin(gdf_metro[['buf_200m','ID']].set_geometry('buf_200m'), gdf_metro[['geometry','ID','Name']].to_crs('epsg:32637'), how='left')

dict_metro=dict()
t=set()
for v in gdf_metro200.ID_left.unique():
    f = set(gdf_metro200[gdf_metro200.ID_left==v]['ID_right'].unique())
    s = set(gdf_metro200[gdf_metro200.ID_right==v]['ID_left'].unique())
    
    if v not in t:
        dict_metro[v] = f.union(s)
        t = t.union(f.union(s))

len(dict_metro)

dict_buf_size=dict()

for v, k in dict_metro.items():
#     print(k)
    dict_buf_size[v] = len(k)

pd.Series(dict_buf_size).to_frame().shape

(gdf_metro200[['ID_left','buf_200m',]]

df_metro_buf = pd.Series(dict_buf_size).to_frame().join(gdf_metro200[['ID_left','buf_200m','Name']].dissolve('ID_left'))

df_metro_buf['Name_fin'] = df_metro_buf['Name'].apply(lambda x: x.split(',')[0])

gdf_metro_buf=gpd.GeoDataFrame(df_metro_buf, geometry='buf_200m', crs='epsg:32637')

gdf_metro_buf.to_crs('epsg:4326', inplace=True)

gdf_metro_buf.rename(columns={0:'weight'}, inplace=True)

gdf_metro_buf.to_file('gdf_metro_buf.geojson', driver='GeoJSON')

In [ ]:
бц - 250чел

2 метро

300 метров

700 метров



In [255]:
gdf_metro

NameError: name 'gdf_metro' is not defined

In [253]:
250*9/34

66.17647058823529

In [236]:
# tag='buildings'
# set([pd.Series(i.tags) for i in result.nodes])

In [209]:
result.nodes[5].id

638742556

In [191]:
len(result.nodes)

1262

In [216]:
dict_result = dict()
for x in result.nodes:
    dict_temp = dict()
    for v, k in x.tags.items():
        dict_temp[v]=k
#     if 'building' in dict_temp.keys() and dict_temp['building'] in ['office','commercial']:
    dict_result[x.id]=dict_temp

In [217]:
dict_result 

{303495712: {'amenity': 'fuel',
  'brand': 'Газпром нефть',
  'building': 'yes',
  'fuel:diesel': 'yes',
  'fuel:octane_92': 'yes',
  'fuel:octane_95': 'yes',
  'name': 'Газпромнефть',
  'opening_hours': '24/7',
  'operator': 'ОАО "Газпромнефть-Центр"',
  'payment:cards': 'yes',
  'ref': '103',
  'website': 'https://www.gazprom-neft.ru/'},
 576375541: {'amenity': 'ice_cream',
  'building': 'kiosk',
  'name': 'Мороженое',
  'takeaway': 'only'},
 595448550: {'building': 'kiosk', 'shop': 'newsagent'},
 604244790: {'building:colour': 'white', 'roof:colour': 'darkgrey'},
 607989336: {'building': 'yes', 'man_made': 'dovecote'},
 638742556: {'building': 'bunker',
  'name': 'бомбоубежище',
  'source': 'Wikimapia'},
 672992117: {'amenity': 'ice_cream',
  'building': 'kiosk',
  'building:levels': '1'},
 727537366: {'amenity': 'fuel',
  'building': 'yes',
  'fuel:diesel': 'yes',
  'fuel:lpg': 'yes',
  'fuel:octane_92': 'yes',
  'fuel:octane_95': 'yes',
  'name': 'Energy',
  'operator': 'МосОЙЛ',


In [187]:
df_bc

,permalink,name,address,lat,lon,rubric,url,phones,chain
14,69818458201,На Стремянском,"Россия, Москва, Стремянный переулок, 26",55.727683,37.629827,Бизнес-центр,NaN,NaN,NaN
69,57000109310,Stone Савеловская,"Россия, Москва, улица Двинцев, 3",55.797205,37.595393,Бизнес-центр,NaN,NaN,NaN
80,57031090465,Синица плаза,"Россия, Москва, 2-я Синичкина улица, 9Ас3",55.761695,37.718175,Бизнес-центр,NaN,+7 (926) 773-28-66,NaN
231,57471922485,Разгуляй офис,"Россия, Москва, Новая Басманная улица, 28, стр. 2",55.769114,37.666863,Бизнес-центр,NaN,+7 (495) 150-08-14,NaN
360,61312863456,Загорье,"Россия, Москва, Загорьевская улица, 10, корп. 4",55.578028,37.676787,Бизнес-центр,NaN,+7 (495) 675-71-11;+7 (495) 675-32-73;+7 (495)...,NaN
...,...,...,...,...,...,...,...,...,...
45159,14964342902,Мельницкий,"Россия, Москва, Мельницкий переулок, 1",55.753944,37.658266,Бизнес-центр,NaN,+7 (495) 640-70-97;+7 (985) 222-04-24,NaN
45166,14988398874,Тэн Группа,"Россия, Москва, Авиамоторная улица, 10, корп. 2",55.754299,37.714590,Бизнес-центр,NaN,NaN,NaN
45315,15299856864,Бизнес-центр,"Россия, Москва, 1-я Тверская-Ямская улица, 6",55.771575,37.593965,Бизнес-центр,NaN,NaN,NaN
45319,15324708379,Серпуховской двор,"Россия, Москва, Большая Тульская улица, 10, ст...",55.707863,37.619186,Бизнес-центр,NaN,+7 (495) 232-21-80;+7 (495) 737-72-02,NaN


In [174]:
gpd.GeoDataFrame([ Point(x.lon, x.lat) for x in result.nodes if x.tags['building'] in ['office','yes','commercial'] ] , columns=['geometry')


ValueError: Shape of passed values is (232, 1), indices imply (232, 2)